In [12]:
# Import required libraries
import pandas as pd
import numpy as np
import spacy
import re
import requests
from bs4 import BeautifulSoup
from textblob import TextBlob
from sklearn.feature_extraction.text import TfidfVectorizer, ENGLISH_STOP_WORDS
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
import umap
import warnings
from tqdm import tqdm
import time
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud
from io import BytesIO
import boto3
from spacy.matcher import Matcher

# Download required NLTK data
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')

warnings.filterwarnings('ignore')

# Load SpaCy model for Named Entity Recognition
NER = spacy.load("en_core_web_sm")

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/philipleong/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/philipleong/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/philipleong/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/philipleong/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/philipleong/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [20]:
s3_client = boto3.client(
    's3',
    aws_access_key_id='',
    aws_secret_access_key=''
)

bucket_name = 'bia-data'
news_file_key = 'news_excerpts_parsed.xlsx'

def load_excel_from_s3(bucket, file_key):
    """
    Downloads an Excel file from S3 and loads it into a Pandas DataFrame.
    """
    try:
        # Fetch the file from S3
        file_obj = s3_client.get_object(Bucket=bucket, Key=file_key)
        
        # Read the file into a Pandas DataFrame
        df = pd.read_excel(BytesIO(file_obj['Body'].read()))
        
        print(f"✅ Loaded {file_key}: {df.shape[0]} rows, {df.shape[1]} columns")
        return df

    except Exception as e:
        print(f"❌ Failed to load {file_key} from S3: {e}")
        return None  # Return None if the file is missing or an error occurs

# Load Datasets from S3
news_excerpts = load_excel_from_s3(bucket_name, news_file_key)

# Check if DataFrames are loaded successfully
if news_excerpts is not None:
    print("✅ All datasets loaded successfully!")
else:
    print("⚠️ Error in loading datasets")

✅ Loaded news_excerpts_parsed.xlsx: 1509 rows, 2 columns
✅ All datasets loaded successfully!


In [24]:
COUNTRIES = {
    'United States': ['United States', 'US', 'USA', 'U.S.', 'America', 'Washington', 'New York', 'California'],
    'China': ['China', 'Chinese', 'Beijing', 'Shanghai', 'Shenzhen', 'Hong Kong'],
    'Singapore': ['Singapore', 'Singaporean'],
    'United Kingdom': ['UK', 'Britain', 'England', 'London', 'United Kingdom', 'British'],
    'European Union': ['EU', 'European Union', 'Brussels', 'Europe', 'European'],
    'Japan': ['Japan', 'Japanese', 'Tokyo', 'Osaka'],
    'India': ['India', 'Indian', 'New Delhi', 'Mumbai', 'Bangalore'],
    'Australia': ['Australia', 'Australian', 'Sydney', 'Melbourne', 'Canberra'],
    'Canada': ['Canada', 'Canadian', 'Toronto', 'Ottawa', 'Vancouver'],
    'Germany': ['Germany', 'German', 'Berlin', 'Frankfurt'],
    'France': ['France', 'French', 'Paris'],
    'Russia': ['Russia', 'Russian', 'Moscow'],
    'South Korea': ['South Korea', 'Korean', 'Seoul'],
    'Brazil': ['Brazil', 'Brazilian', 'Sao Paulo', 'Rio'],
    'Indonesia': ['Indonesia', 'Indonesian', 'Jakarta'],
    'Thailand': ['Thailand', 'Thai', 'Bangkok'],
    'Vietnam': ['Vietnam', 'Vietnamese', 'Hanoi'],
    'Malaysia': ['Malaysia', 'Malaysian', 'Kuala Lumpur']
}

def add_id_column(df, column_name='Article Id', start_from=1):
    # Create a copy to avoid modifying the original DataFrame
    result = df.copy()
    
    # Add ID column
    result.insert(0, column_name, range(start_from, len(df) + start_from))
    
    return result

def clean_text(text):
    """Clean and preprocess text with enhanced stop word removal."""
    # Create custom stop words set
    custom_stop_words = set(stopwords.words('english')).union(ENGLISH_STOP_WORDS)
    custom_stop_words.update(['said', 'says', 'tell', 'told'])
    
    # Lowercase and remove punctuation
    text = re.sub(r'[^\w\s]', '', text.lower())
    
    # Remove stop words
    words = text.split()
    cleaned_words = [word for word in words if word not in custom_stop_words]
    
    return ' '.join(cleaned_words)

stop_words = set(stopwords.words('english')).union(ENGLISH_STOP_WORDS)
stop_words.update(['said', 'says', 'tell', 'told'])  # Custom stop words

def preprocess_text(text):
    """Clean and preprocess text."""
    text = str(text).lower()
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    tokens = word_tokenize(text)
    tokens = [token for token in tokens if token not in stop_words]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    return ' '.join(tokens)

# Extract country mentions
def extract_countries_from_text(text):
    """Extract countries and their counts from text."""
    text = text.lower()
    country_mentions = []
    for country, keywords in COUNTRIES.items():
        for keyword in keywords:
            if keyword.lower() in text:
                country_mentions.append(country)
                break
    if country_mentions:
        country_counts = Counter(country_mentions)
        most_mentioned = max(country_counts.items(), key=lambda x: x[1])[0]
        return {'primary_country': most_mentioned, 'all_countries': dict(country_counts)}
    return {'primary_country': None, 'all_countries': {}}

# Sentiment analysis
def get_sentiment(text):
    """Calculate sentiment polarity."""
    blob = TextBlob(str(text))
    return round(blob.sentiment.polarity, 4)

# Extract cluster keywords
def get_cluster_terms_frequencies(vectorizer, kmeans, tfidf_matrix, n_terms=10):
    """Get top terms and their frequencies for each cluster."""
    terms = vectorizer.get_feature_names_out()
    clusters = kmeans.labels_
    cluster_term_freq = []
    for cluster_idx in range(kmeans.n_clusters):
        cluster_docs = tfidf_matrix[clusters == cluster_idx]
        avg_tfidf = cluster_docs.mean(axis=0).A1
        top_indices = avg_tfidf.argsort()[-n_terms:][::-1]
        for idx in top_indices:
            cluster_term_freq.append({'Cluster': cluster_idx, 'Term': terms[idx], 'Frequency': avg_tfidf[idx]})
    return pd.DataFrame(cluster_term_freq)

def process_articles(df):
    """Clean, analyze, and cluster articles."""
    print("Cleaning and preprocessing text...")
    tqdm.pandas()
    df['cleaned_text'] = df['Text'].progress_apply(preprocess_text)

    print("Extracting countries and calculating sentiment...")
    countries_data = df['Text'].progress_apply(extract_countries_from_text)
    df['primary_country'] = countries_data.apply(lambda x: x['primary_country'])
    df['all_countries'] = countries_data.apply(lambda x: x['all_countries'])
    df['sentiment'] = df['Text'].progress_apply(get_sentiment)

    print("Clustering articles...")
    vectorizer = TfidfVectorizer(max_features=1000, stop_words='english')
    tfidf_matrix = vectorizer.fit_transform(df['cleaned_text'])
    kmeans = KMeans(n_clusters=5, random_state=42)
    df['cluster'] = kmeans.fit_predict(tfidf_matrix)
    term_freq_df = get_cluster_terms_frequencies(vectorizer, kmeans, tfidf_matrix)
    return df, term_freq_df

def extract_relationships(doc, nlp):
    """Extract relationships using rule-based and dependency-based methods."""
    relationships = []
    
    # Initialize spaCy Matcher
    matcher = Matcher(nlp.vocab)
    
    # Rule 1: X and Y (compound relationships)
    pattern_and = [{"POS": "PROPN"}, {"LOWER": "and"}, {"POS": "PROPN"}]
    matcher.add("AND_PATTERN", [pattern_and])
    
    # Rule 2: X's Y (possessive relationships)
    pattern_possessive = [{"POS": "PROPN"}, {"LOWER": "'s"}, {"POS": "NOUN"}]
    matcher.add("POSSESSIVE_PATTERN", [pattern_possessive])
    
    # Rule 3: X is Y (appositive relationships)
    pattern_appositive = [{"POS": "PROPN"}, {"LOWER": "is"}, {"POS": "PROPN"}]
    matcher.add("APPOSITIVE_PATTERN", [pattern_appositive])
    
    # Rule 4: X who is Y (relative clause relationships)
    pattern_relative_clause = [{"POS": "PROPN"}, {"LOWER": "who"}, {"LOWER": "is"}, {"POS": "NOUN"}]
    matcher.add("RELATIVE_CLAUSE_PATTERN", [pattern_relative_clause])
    
    # Rule 5: X quickly Y (verb phrases with modifiers)
    pattern_verb_phrase = [{"POS": "PROPN"}, {"POS": "ADV"}, {"POS": "VERB"}]
    matcher.add("VERB_PHRASE_PATTERN", [pattern_verb_phrase])
    
    # Find matches in the document
    matches = matcher(doc)
    for match_id, start, end in matches:
        span = doc[start:end]
        match_label = nlp.vocab.strings[match_id]
        relationships.append(("Rule-based", span.text))
    
    # Dependency-based extraction
    for token in doc:
        if token.dep_ in ("nsubj", "dobj", "pobj"):  # Focus on subject, object relationships
            if token.head.pos_ in ("VERB", "NOUN"):
                if token.dep_ == "nsubj":
                    rel_type = "SUBJECT_OF"
                elif token.dep_ == "dobj":
                    rel_type = "OBJECT_OF"
                elif token.dep_ == "pobj":
                    rel_type = "OBJECT_OF_PREPOSITION"
                relationships.append((rel_type, f"{token.text} -> {token.head.text}"))
    
    return relationships

def save_relationships_to_excel(relationships, file_name="relationships_output.xlsx"):
    """Save relationships to an Excel file."""
    df = pd.DataFrame(relationships, columns=["Relationship Type", "Source Entity", "Target Entity", 
                                              "Cluster Number", "Distance to Centroid"])
    # Create an in-memory Excel file
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name="Extracted Relationships", index=False)
    output.seek(0)  # Reset file pointer

    s3_folder = 'exports'
    s3_file_key = f"{s3_folder}/{file_name}"

    # Upload the Excel file to S3
    s3_client.put_object(Bucket=bucket_name, Key=s3_file_key, Body=output.getvalue())
    
    print(f"🚀 Uploaded {file_name} to s3://{bucket_name}/{s3_file_key}")
    return df

def k_means_clustering_and_extract_relationships(df, text_column, n_clusters=5):
    nlp = spacy.load("en_core_web_sm")
    """Perform KMeans clustering and extract relationships."""
    # Clean text
    df[text_column] = df[text_column].apply(clean_text)
    
    # Extract text
    documents = df[text_column].dropna().tolist()
    
    # TF-IDF Vectorization
    vectorizer = TfidfVectorizer()
    vectorized_texts = vectorizer.fit_transform(documents)
    
    # Perform KMeans clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(vectorized_texts)
    
    # Calculate distances to cluster centroids
    distances = kmeans.transform(vectorized_texts)
    
    all_relationships = []  # To store all extracted relationships
    
    # Extract relationships from each cluster
    for cluster in range(n_clusters):
        cluster_docs = []
        cluster_distances = []
        
        # Collect documents and their distances for this cluster
        for idx, (doc, label) in enumerate(zip(documents, cluster_labels)):
            if label == cluster:
                cluster_docs.append(doc)
                cluster_distances.append(distances[idx][cluster])
        
        cluster_text = ' '.join(cluster_docs)
        
        # Process text with spaCy
        doc = nlp(cluster_text)
        
        # Extract relationships
        relationships = extract_relationships(doc, nlp)
        
        # Organize relationships with source and target entities
        for rel_type, rel_text in relationships:
            if "->" in rel_text:  # Dependency-based relationships
                source, target = rel_text.split(" -> ")
                # Use the minimum distance to centroid for this relationship
                min_distance = min(cluster_distances)
                all_relationships.append([rel_type, source, target, cluster, min_distance])
            else:  # Rule-based relationships
                entities = rel_text.split()
                if len(entities) >= 2:
                    source, target = entities[0], entities[-1]
                    min_distance = min(cluster_distances)
                    all_relationships.append([rel_type, source, target, cluster, min_distance])
        
    # Save relationships to Excel
    df = save_relationships_to_excel(all_relationships)
    return df
    
def filter_words(node_list, count_dict, num_keywords, threshold=3):
    '''returns the top num_keywords occuring keywords, default 50'''
        
    # set dynamic threshold, limit number of keywords to 50
    filtered = node_list.copy()
    num_keywords = min(num_keywords,  0.1 * len(node_list))
        
    while (len(filtered) > num_keywords or len(filtered) == 0):
        for node in filtered:
            if count_dict.get(node, 0) < threshold:
                    filtered.remove(node)
        threshold += 1
            
    return filtered

def get_relationship_key_words(df, num_clusters=5, num_keywords=50):
    '''returns top num_keywords keywords per cluster'''
    cluster_keywords = []
    for i in range(num_clusters):
        cluster_df = df[df['Cluster Number'] == i]
        source_names = cluster_df['Source Entity'].to_list()
        source_names = list(dict.fromkeys(source_names))
        target_names = cluster_df['Target Entity'].to_list()
        target_names = list(dict.fromkeys(target_names))
        
        source_count_dict = {}
        target_count_dict = {}

        for x in source_names:
            count = source_count_dict.get(x, 0)
            count += 1
            source_count_dict[x]=count
        
        source_keywords = filter_words(source_names, source_count_dict, num_keywords)
        target_keywords = filter_words(target_names, target_count_dict, num_keywords)
        cluster_keywords.extend([{'term': x, 'type': 'source', 'cluster': i} for x in source_keywords])
        cluster_keywords.extend([{'term': x, 'type': 'target', 'cluster': i} for x in target_keywords])
        
    return cluster_keywords


def convert_and_save_keywords(df):
    '''Save top 50 keywords per cluster to an in-memory Excel file and return it for S3 upload'''
    keywords = get_relationship_key_words(df)  # Extract relationship keywords
    key_df = pd.DataFrame(keywords)

    # Create an in-memory Excel file
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        key_df.to_excel(writer, sheet_name='Relationship_Keywords_By_Cluster', index=False)
    output.seek(0)

    return output

# Export results to Excel and upload to S3
def export_and_upload_results(processed_df, term_freq_df, output_file, bucket_name, s3_folder):
    """Save results to Excel and upload to S3."""
    # Save to Excel
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        processed_df.to_excel(writer, sheet_name='Country Analysis', index=False)
        term_freq_df.to_excel(writer, sheet_name='Term Frequencies', index=False)
    output.seek(0)

    # Upload to S3
    s3_file_key = f"{s3_folder}/{output_file}"
    s3_client.put_object(Bucket=bucket_name, Key=s3_file_key, Body=output.getvalue())
    print(f"🚀 Uploaded {output_file} to s3://{bucket_name}/{s3_file_key}")

def save_combined_excel_to_s3(processed_df, term_freq_df, rel_df, keywords_df, bucket_name, s3_folder, file_name="combined_analysis.xlsx"):
    output = BytesIO()
    with pd.ExcelWriter(output, engine='openpyxl') as writer:
        processed_df.to_excel(writer, sheet_name="Country Analysis", index=False)
        term_freq_df.to_excel(writer, sheet_name="Term Frequencies", index=False)
        rel_df.to_excel(writer, sheet_name="Extracted Relationships", index=False)
        keywords_df.to_excel(writer, sheet_name="Relationship_Keywords_By_Cluster", index=False)
    
    output.seek(0)

    s3_file_key = f"{s3_folder}/{file_name}"

    s3_client.put_object(Bucket=bucket_name, Key=s3_file_key, Body=output.getvalue())

    print(f"🚀 Successfully uploaded {file_name} to s3://{bucket_name}/{s3_file_key}")

    return s3_file_key

def main():
    print("Starting analysis...")
    processed_df, term_freq_df = process_articles(news_excerpts)

    # Prepare data for export
    output_file = 'text_analysis_for_tableau.xlsx'
    s3_folder = 'exports'

    export_and_upload_results(processed_df, term_freq_df, output_file, bucket_name, s3_folder)

    df = add_id_column(news_excerpts)

    # Rule-based relationships extraction
    print("\nExtracting relationships...")
    rel_df = k_means_clustering_and_extract_relationships(df, 'Text')

    keywords_df = pd.DataFrame(get_relationship_key_words(rel_df))

    # Extract top 50 occurring source & target relationships
    top_50_excel = convert_and_save_keywords(rel_df)

    # Define S3 upload path for relationship keywords file
    relationship_keywords_file = "relationship_keywords_output.xlsx"
    s3_file_key = f"{s3_folder}/{relationship_keywords_file}"

    # Upload the relationship keywords file to S3
    s3_client.put_object(Bucket=bucket_name, Key=s3_file_key, Body=top_50_excel.getvalue())
    print(f"🚀 Uploaded {relationship_keywords_file} to s3://{bucket_name}/{s3_file_key}")


    # Save combined Excel and upload to S3
    combined_file_name = "combined_analysis.xlsx"
    s3_path = save_combined_excel_to_s3(processed_df, term_freq_df, rel_df, keywords_df, bucket_name, s3_folder, combined_file_name)

    print(f"✅ Combined Excel file uploaded to: s3://{bucket_name}/{s3_path}")
                                          
    print("Analysis complete!")

if __name__ == "__main__":
    main()

Starting analysis...
Cleaning and preprocessing text...


100%|███████████████████████████████████| 1509/1509 [00:00<00:00, 3371.93it/s]


Extracting countries and calculating sentiment...


100%|███████████████████████████████████| 1509/1509 [00:00<00:00, 3947.35it/s]


Clustering articles...
🚀 Uploaded text_analysis_for_tableau.xlsx to s3://bia-data/exports/text_analysis_for_tableau.xlsx

Extracting relationships...
🚀 Uploaded relationships_output.xlsx to s3://bia-data/exports/relationships_output.xlsx
🚀 Uploaded relationship_keywords_output.xlsx to s3://bia-data/exports/relationship_keywords_output.xlsx
🚀 Successfully uploaded combined_analysis.xlsx to s3://bia-data/exports/combined_analysis.xlsx
✅ Combined Excel file uploaded to: s3://bia-data/exports/combined_analysis.xlsx
Analysis complete!
